In [ ]:
from utils import *

In [ ]:
from azureml.core import Workspace

# Configure experiment
ws = Workspace.from_config()

In [ ]:
# Create or get training cluster
aml_cluster = get_aml_cluster(ws, cluster_name="cpu-cluster")
aml_cluster.wait_for_completion(show_output=True)

In [ ]:
# Create a run configuration
run_conf = get_run_config(['numpy', 'pandas', 'scikit-learn', 'tensorflow'])

In [ ]:
from azureml.core import Dataset

dataset = Dataset.get_by_name(ws, name='titanic')
data_in = dataset.as_named_input('titanic')

In [ ]:
from azureml.core import Datastore
from azureml.pipeline.core import PipelineData

datastore = Datastore.get(ws, datastore_name="mldata")
results = PipelineData('results', datastore=datastore)

In [ ]:
from azureml.pipeline.core import PipelineParameter
from azureml.pipeline.steps import ParallelRunConfig

parallel_run_config = ParallelRunConfig(
    entry_script='score.py',
    source_directory='code',
    mini_batch_size=5,
    error_threshold=10,
    output_action="append_row",
    append_row_file_name="parallel_run_step.txt",
    environment=run_conf.environment,
    compute_target=aml_cluster,
    process_count_per_node=2,
    node_count=2)


In [ ]:
from azureml.pipeline.steps import ParallelRunStep

parallelrun_step = ParallelRunStep(
    name="ScoreParallel",
    parallel_run_config=parallel_run_config,
    inputs=[data_in],
    output=results,
    allow_reuse=True
)


In [ ]:
from azureml.pipeline.core import Pipeline

pipeline = Pipeline(ws, steps=[parallelrun_step])

In [ ]:
pipeline.validate()

In [ ]:
from azureml.core import Experiment

exp = Experiment(ws, "azureml-parallel-pipeline")
run = exp.submit(pipeline)

In [ ]:
from azureml.widgets import RunDetails
RunDetails(run).show()